In [33]:
import hashlib

import pandas as pd
import numpy as np
import re
import json

from requests.utils import set_environ

In [6]:
with open("../fine_art_ner/datasets/002.txt", "r") as f:
        data = f.read()
        # print(data)
        # data = json.loads(data)
        data = data.replace("\n```", "").replace("json", "").replace("```\n", "")
        print(data)

{
    "sentence": "Acrylic paint - 'Starry Nite' by Vincent van Gohg; 1889, 73cm x 92cm",
    "entities": [
        (35, 51, "ARTIST"),
        (54, 58, "DATE"),
        (0, 13, "MEDIUM"),
        (16, 28, "ART_TITLE"),
        (60, 70, "DIMENSION")
    ]
}
{
    "sentence": "The Persistence of Memory, Salvador Dali - oil on canvas; 1931, 24x33cm",
    "entities": [
        (29, 42, "ARTIST"),
        (46, 60, "MEDIUM"),
        (62, 66, "DATE"),
        (0, 26, "ART_TITLE"),
        (68, 75, "DIMENSION")
    ]
}
{
    "sentence": "Leonardo da Vincy - 'Mona Lisa'; 77cm x 53cm, 1503, oil paint",
    "entities": [
        (0, 17, "ARTIST"),
        (20, 30, "ART_TITLE"),
        (33, 43, "DIMENSION"),
        (45, 49, "DATE"),
        (51, 60, "MEDIUM")
    ]
}
{
    "sentence": "Watercolor, 'The Great Wave off Kanagwa' - Hokusai; 1831, 25.7cm x 37.8cm",
    "entities": [
        (42, 49, "ARTIST"),
        (52, 56, "DATE"),
        (0, 10, "MEDIUM"),
        (12, 39, "ART_TITLE"),
     

In [7]:
data = data.replace("}", "},")
print(data)

{
    "sentence": "Acrylic paint - 'Starry Nite' by Vincent van Gohg; 1889, 73cm x 92cm",
    "entities": [
        (35, 51, "ARTIST"),
        (54, 58, "DATE"),
        (0, 13, "MEDIUM"),
        (16, 28, "ART_TITLE"),
        (60, 70, "DIMENSION")
    ]
},
{
    "sentence": "The Persistence of Memory, Salvador Dali - oil on canvas; 1931, 24x33cm",
    "entities": [
        (29, 42, "ARTIST"),
        (46, 60, "MEDIUM"),
        (62, 66, "DATE"),
        (0, 26, "ART_TITLE"),
        (68, 75, "DIMENSION")
    ]
},
{
    "sentence": "Leonardo da Vincy - 'Mona Lisa'; 77cm x 53cm, 1503, oil paint",
    "entities": [
        (0, 17, "ARTIST"),
        (20, 30, "ART_TITLE"),
        (33, 43, "DIMENSION"),
        (45, 49, "DATE"),
        (51, 60, "MEDIUM")
    ]
},
{
    "sentence": "Watercolor, 'The Great Wave off Kanagwa' - Hokusai; 1831, 25.7cm x 37.8cm",
    "entities": [
        (42, 49, "ARTIST"),
        (52, 56, "DATE"),
        (0, 10, "MEDIUM"),
        (12, 39, "ART_TITLE"),
  

In [9]:
data = f"[{data}]"
print(data)

[{
    "sentence": "Acrylic paint - 'Starry Nite' by Vincent van Gohg; 1889, 73cm x 92cm",
    "entities": [
        (35, 51, "ARTIST"),
        (54, 58, "DATE"),
        (0, 13, "MEDIUM"),
        (16, 28, "ART_TITLE"),
        (60, 70, "DIMENSION")
    ]
},
{
    "sentence": "The Persistence of Memory, Salvador Dali - oil on canvas; 1931, 24x33cm",
    "entities": [
        (29, 42, "ARTIST"),
        (46, 60, "MEDIUM"),
        (62, 66, "DATE"),
        (0, 26, "ART_TITLE"),
        (68, 75, "DIMENSION")
    ]
},
{
    "sentence": "Leonardo da Vincy - 'Mona Lisa'; 77cm x 53cm, 1503, oil paint",
    "entities": [
        (0, 17, "ARTIST"),
        (20, 30, "ART_TITLE"),
        (33, 43, "DIMENSION"),
        (45, 49, "DATE"),
        (51, 60, "MEDIUM")
    ]
},
{
    "sentence": "Watercolor, 'The Great Wave off Kanagwa' - Hokusai; 1831, 25.7cm x 37.8cm",
    "entities": [
        (42, 49, "ARTIST"),
        (52, 56, "DATE"),
        (0, 10, "MEDIUM"),
        (12, 39, "ART_TITLE"),
 

In [18]:
import ast

data = ast.literal_eval(data)

In [22]:
data

[{'sentence': "Acrylic paint - 'Starry Nite' by Vincent van Gohg; 1889, 73cm x 92cm",
  'entities': [(35, 51, 'ARTIST'),
   (54, 58, 'DATE'),
   (0, 13, 'MEDIUM'),
   (16, 28, 'ART_TITLE'),
   (60, 70, 'DIMENSION')]},
 {'sentence': 'The Persistence of Memory, Salvador Dali - oil on canvas; 1931, 24x33cm',
  'entities': [(29, 42, 'ARTIST'),
   (46, 60, 'MEDIUM'),
   (62, 66, 'DATE'),
   (0, 26, 'ART_TITLE'),
   (68, 75, 'DIMENSION')]},
 {'sentence': "Leonardo da Vincy - 'Mona Lisa'; 77cm x 53cm, 1503, oil paint",
  'entities': [(0, 17, 'ARTIST'),
   (20, 30, 'ART_TITLE'),
   (33, 43, 'DIMENSION'),
   (45, 49, 'DATE'),
   (51, 60, 'MEDIUM')]},
 {'sentence': "Watercolor, 'The Great Wave off Kanagwa' - Hokusai; 1831, 25.7cm x 37.8cm",
  'entities': [(42, 49, 'ARTIST'),
   (52, 56, 'DATE'),
   (0, 10, 'MEDIUM'),
   (12, 39, 'ART_TITLE'),
   (58, 71, 'DIMENSION')]},
 {'sentence': "Pablo Picaso, 'Guernica' - 1937, 349cm x 776cm; oil on canvas",
  'entities': [(0, 13, 'ARTIST'),
   (16, 25, 'A

In [38]:
# Convert data to format that is acceptable for dataframe.
formatted_data = []

for i, doc in enumerate(data):

    # Compute hash of document.
    doc_string = json.dumps(doc, sort_keys=True)
    doc_hash = hashlib.md5(doc_string.encode()).hexdigest()

    for entity in doc["entities"]:

        start_idx = entity[0]
        end_idx = entity[1]
        entity_type = entity[2]

        entity_dict = dict(
            sentence=doc["sentence"],
            start_idx=start_idx,
            end_idx=end_idx,
            entity_type=entity_type,
        )
        entity_string = json.dumps(entity_dict, sort_keys=True)
        entity_hash = hashlib.md5(entity_string.encode()).hexdigest()

        formatted_doc = dict(
            doc_id=i,
            doc_hash=doc_hash,
            entity_hash=entity_hash,
            **entity_dict
        )

        formatted_data.append(formatted_doc)

formatted_data

[{'doc_id': 0,
  'doc_hash': '01484f84b35d80b2c0c35cb57a700429',
  'entity_hash': '3fdf93e712914ea84663d6d67b463e4a',
  'sentence': "Acrylic paint - 'Starry Nite' by Vincent van Gohg; 1889, 73cm x 92cm",
  'start_idx': 35,
  'end_idx': 51,
  'entity_type': 'ARTIST'},
 {'doc_id': 0,
  'doc_hash': '01484f84b35d80b2c0c35cb57a700429',
  'entity_hash': 'adc175000120f2a768ffa3bc3474b8e3',
  'sentence': "Acrylic paint - 'Starry Nite' by Vincent van Gohg; 1889, 73cm x 92cm",
  'start_idx': 54,
  'end_idx': 58,
  'entity_type': 'DATE'},
 {'doc_id': 0,
  'doc_hash': '01484f84b35d80b2c0c35cb57a700429',
  'entity_hash': '3b9dee514918fcfe35c4188ee32d341b',
  'sentence': "Acrylic paint - 'Starry Nite' by Vincent van Gohg; 1889, 73cm x 92cm",
  'start_idx': 0,
  'end_idx': 13,
  'entity_type': 'MEDIUM'},
 {'doc_id': 0,
  'doc_hash': '01484f84b35d80b2c0c35cb57a700429',
  'entity_hash': 'e20801a0ddc2b0a1fda7dd12f20d529f',
  'sentence': "Acrylic paint - 'Starry Nite' by Vincent van Gohg; 1889, 73cm x 92

In [39]:
df = pd.DataFrame(formatted_data)
df

,doc_id,doc_hash,entity_hash,sentence,start_idx,end_idx,entity_type
0,0,01484f84b35d80b2c0c35cb57a700429,3fdf93e712914ea84663d6d67b463e4a,Acrylic paint - 'Starry Nite' by Vincent van G...,35,51,ARTIST
1,0,01484f84b35d80b2c0c35cb57a700429,adc175000120f2a768ffa3bc3474b8e3,Acrylic paint - 'Starry Nite' by Vincent van G...,54,58,DATE
2,0,01484f84b35d80b2c0c35cb57a700429,3b9dee514918fcfe35c4188ee32d341b,Acrylic paint - 'Starry Nite' by Vincent van G...,0,13,MEDIUM
3,0,01484f84b35d80b2c0c35cb57a700429,e20801a0ddc2b0a1fda7dd12f20d529f,Acrylic paint - 'Starry Nite' by Vincent van G...,16,28,ART_TITLE
4,0,01484f84b35d80b2c0c35cb57a700429,c39139ba13ede83adac7d8abbbc68718,Acrylic paint - 'Starry Nite' by Vincent van G...,60,70,DIMENSION
...,...,...,...,...,...,...,...
10360,2074,4fa49f4d5d7e0d51bfea391141a88ddf,1c5f2d58aee9ff6d333ce4674c606145,"Vincent van Gogh - 1889; 'Irises', oil on canv...",0,16,ARTIST
10361,2074,4fa49f4d5d7e0d51bfea391141a88ddf,f494c9a82d2e613aaae545345deb5491,"Vincent van Gogh - 1889; 'Irises', oil on canv...",19,23,DATE
10362,2074,4fa49f4d5d7e0d51bfea391141a88ddf,60430cd5ebeb080cf243f182c85302e6,"Vincent van Gogh - 1889; 'Irises', oil on canv...",25,32,ART_TITLE
10363,2074,4fa49f4d5d7e0d51bfea391141a88ddf,d5df6b8a69caf6837fbc44e53d352c7c,"Vincent van Gogh - 1889; 'Irises', oil on canv...",34,48,MEDIUM


In [58]:
test_sentence = "Tempera on panel: 'The Birth of Venus' - Sandro Botticelli, 1486; 172.5cm x 278.9cm"
test_sentence[42:60]

'andro Botticelli, '

In [40]:
# df.to_csv("datasets/CSV/002.csv", index=True)

In [49]:
# Convert to CoNLL format.
# for doc in data:
#     print(doc)
#     sentence = doc["sentence"]
#     pattern = r"\w+"
#     result = re.search(pattern, sentence)
#     print(result.gro)
#     break

{'sentence': "Acrylic paint - 'Starry Nite' by Vincent van Gohg; 1889, 73cm x 92cm", 'entities': [(35, 51, 'ARTIST'), (54, 58, 'DATE'), (0, 13, 'MEDIUM'), (16, 28, 'ART_TITLE'), (60, 70, 'DIMENSION')]}
()


In [52]:
# Function to convert to CoNLL format
def convert_to_conll(data):
    # Tokenize sentence
    sentence = data["sentence"]
    print(sentence)
    tokens = re.findall(r"\S+", sentence)  # Splits by spaces, keeps punctuation
    print(tokens)

    # Initialize labels for each token
    labels = ["O"] * len(tokens)

    # Assign entity tags to tokens
    for start, end, entity_type in data["entities"]:
        # Find tokens overlapping with entity character range
        entity_tokens = []
        for i, t in enumerate(tokens):
            print(i, t)
            if start <= sentence.index(t, start) < end:
                entity_tokens.append((i, t))

        entity_tokens = [(i, token) for i, token in enumerate(tokens)
                         if start <= sentence.index(token, start) < end]

        # Assign B- and I- tags
        if entity_tokens:
            labels[entity_tokens[0][0]] = f"B-{entity_type}"  # First token of entity
            for i, _ in entity_tokens[1:]:
                labels[i] = f"I-{entity_type}"  # Remaining tokens

    # Format as CoNLL
    conll_lines = [f"{token} {label}" for token, label in zip(tokens, labels)]
    return "\n".join(conll_lines)


# Convert our data
for doc in data:
    conll_output = convert_to_conll(doc)
    print(conll_output)
    break

Acrylic paint - 'Starry Nite' by Vincent van Gohg; 1889, 73cm x 92cm
['Acrylic', 'paint', '-', "'Starry", "Nite'", 'by', 'Vincent', 'van', 'Gohg;', '1889,', '73cm', 'x', '92cm']


ValueError: substring not found

In [55]:
test_str = "Acrylic paint - 'Starry Nite' by Vincent van Gohg; 1889, 73cm x 92cm"
test_str.index("Acrylic", 5)

ValueError: substring not found

In [65]:
import re

# Original data
test_data = {
    "sentence": "Acrylic paint - 'Starry Nite' by Vincent van Gohg; 1889, 73cm x 92cm",
    "entities": [
        (35, 51, "ARTIST"),
        (54, 58, "DATE"),
        (0, 13, "MEDIUM"),
        (16, 28, "ART_TITLE"),
        (60, 70, "DIMENSION"),
    ],
}


# Function to map characters to tokens and match character indices
def convert_to_conll(data):
    # Tokenize the sentence
    sentence = data["sentence"]
    tokens = re.findall(r"\S+", sentence)  # Tokenize the text by splitting on spaces
    token_start_indices = []
    offset = 0

    # Map tokens with their starting character indices
    for token in tokens:
        start_idx = sentence.find(token, offset)
        token_start_indices.append((start_idx, start_idx + len(token)))
        offset = start_idx + len(token)  # Move offset to prevent duplicate matching

    # Initialize labels for tokens
    labels = ["O"] * len(tokens)

    # Loop through entities and match indices to the token ranges
    for start, end, entity_type in data["entities"]:
        for i, (token_start, token_end) in enumerate(token_start_indices):
            # Check if the token overlaps with the entity's character range
            if token_start >= start and token_start < end:
                if labels[i] == "O":  # Only tag when label isn't already set
                    if token_start == start:
                        labels[i] = f"B-{entity_type}"
                    else:
                        labels[i] = f"I-{entity_type}"
                else:
                    labels[i] = f"I-{entity_type}"  # Handle continuation tags

    # Generate CoNLL format lines
    conll_lines = [f"{token} {label}" for token, label in zip(tokens, labels)]
    return "\n".join(conll_lines)


# Convert the data to CoNLL format
for doc in data:
    conll_output = convert_to_conll(doc)
    with open("datasets/conll/test.txt", "a") as f:
        f.write(conll_output + "\n\n")

In [62]:
data[-3]

{'sentence': "Tempera on panel: 'The Birth of Venus' - Sandro Botticelli, 1486; 172.5cm x 278.9cm",
 'entities': [(0, 17, 'MEDIUM'),
  (19, 39, 'ART_TITLE'),
  (42, 60, 'ARTIST'),
  (62, 66, 'DATE'),
  (68, 83, 'DIMENSION')]}

In [64]:
test_sentence = "Tempera on panel: 'The Birth of Venus' - Sandro Botticelli, 1486; 172.5cm x 278.9cm"
test_sentence[41:60]

'Sandro Botticelli, '

In [ ]:
# For each word